<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Импорт-данных" data-toc-modified-id="Импорт-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт данных</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Вывод-первых-строк-датафрейма" data-toc-modified-id="Вывод-первых-строк-датафрейма-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Вывод первых строк датафрейма</a></span></li><li><span><a href="#Вывод-основной-информации" data-toc-modified-id="Вывод-основной-информации-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Вывод основной информации</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Токенизация-текста" data-toc-modified-id="Токенизация-текста-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Токенизация текста</a></span></li><li><span><a href="#Создание-эмбедингов" data-toc-modified-id="Создание-эмбедингов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Создание эмбедингов</a></span></li><li><span><a href="#Подготовка-признаков" data-toc-modified-id="Подготовка-признаков-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Подготовка признаков</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с использованием BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка
### Импорт данных
Импортирую необходимые библиотеки

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import torch
import transformers
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

### Загрузка данных
Загружаю данные

In [2]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', index_col=0)

### Вывод первых строк датафрейма
Вывод первых строк

In [3]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


### Вывод основной информации
Вывожу информацию о датафрейме

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Также вывожу процент положительных значений в целевом признаке

In [5]:
data['toxic'].mean()

0.10161213369158527

Наблюдаю дисбаланс данных. Его необходимо будет учесть при обучении

**Вывод раздела.** В данном разделе были импортированы основные билиотеки, загружены данные и просмотрена основная информация о них.

## Обучение
### Подготовка данных
Урежу данные до 159000 для упрощения подбора размера батчей в будущем

In [6]:
data = data.sample(3000, random_state=1).reset_index(drop=True)

Проверяю сохранение пропорций целевого признака

In [7]:
data['toxic'].mean()

0.10633333333333334

С пропорциями всё хорошо.

Указываю путь до модели

In [8]:
path = 'unitary/toxic-bert'

### Токенизация текста
Создаю токенизатор `BertTokenizer` и назначаю максимальную длину токена - 512, так как BERT работает именно с такой длиной токена. Токенизирую весь текст.

In [9]:
tokenizer = transformers.BertTokenizer.from_pretrained(path)

tqdm.pandas(desc='Apply progress')
tokenized = data['text'].progress_apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))

Apply progress:   0%|          | 0/3000 [00:00<?, ?it/s]

Токены имеют разную длину, поэтому в рядах, где длина токена не достигает 512, заполняю токен нулями до необходимой длины. Также создаю маску для важных токенов.

In [10]:
max_len = 512
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

### Создание эмбедингов
Для создания эмбедингов инициализирую модель

In [11]:
model = transformers.BertModel.from_pretrained(path).cuda()

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Указываю размер батча и получаю эмбединги для текста

In [12]:
batch_size = 100
embeddings = []
for i in tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).cuda()
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/30 [00:00<?, ?it/s]

### Подготовка признаков
Собираю эмбединги в матрицу признаков, а также выделяю целевой признак из датафрейма

In [13]:
features = np.concatenate(embeddings)
target = data['toxic']

Делю данные на тренировачную валидационную и тестовую выборки с соотношением 3:1:1

In [14]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features,
    target,
    test_size=0.6,
    random_state=1
)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid,
    target_valid,
    test_size=0.5,
    random_state=1
)

Ищу лучшую модель перебором гиперпараметров

In [15]:
# Переменные для хранения параметров наилучшей модели
best_model = 0
best_model_title = 0
# Начинаю с оценки логистической регрессии, так как она не требует перебора гиперпараметров
model = LogisticRegression(random_state=1, class_weight='balanced', max_iter=1000)
model.fit(features_train, target_train)
# Сохраняю оценку, как лучшую, так как это первая модель
best_score = f1_score(target_valid, model.predict(features_valid))
best_model = model
best_model_title = 'LogisticRegression'
# Дерево решений
# Цикл для перебора гиперпараметров модели дерева решений
for depth in tqdm(range(5, 31, 5), desc='DecisionTreeClassifier (max_depth)', position=0, leave=False):
    for samples_leaf in tqdm(range(1, 51), desc='DecisionTreeClassifier (samples_leaf)', position=1, leave=False):
        model = DecisionTreeClassifier(min_samples_leaf=samples_leaf, max_depth=depth, class_weight='balanced', random_state=1)
        model.fit(features_train, target_train)
        score = f1_score(target_valid, model.predict(features_valid))
        if best_score < score:
            best_model = model
            best_model_title = f'DecisionTreeClassifier(min_samples_leaf={samples_leaf}, max_depth={depth})'
            best_score = score
# Случайный лес
# Цикл для перебора гиперпараметров модели случайного леса
for depth in tqdm(range(5, 31, 5), desc='RandomForestClassifier (max_depth)', position=0, leave=False):
    for est in tqdm(range(100, 501, 50), desc='RandomForestClassifier (n_estimators)', position=1, leave=False):
        model = RandomForestClassifier(n_estimators=est, max_depth=depth, class_weight='balanced', random_state=1)
        model.fit(features_train, target_train)
        score = f1_score(target_valid, model.predict(features_valid))
        if best_score < score:
            best_model = model
            best_model_title = f'RandomForestClassifier(n_estimators={est}, max_depth={depth})'
            best_score = score
# Вывод лучшей модели и её параметров для целевого признака
print(f'Лучшая модель: {best_model_title}')
print(f'F1_SCORE: {best_score}')

DecisionTreeClassifier (max_depth):   0%|          | 0/6 [00:00<?, ?it/s]

DecisionTreeClassifier (samples_leaf):   0%|          | 0/50 [00:00<?, ?it/s]

DecisionTreeClassifier (samples_leaf):   0%|          | 0/50 [00:00<?, ?it/s]

DecisionTreeClassifier (samples_leaf):   0%|          | 0/50 [00:00<?, ?it/s]

DecisionTreeClassifier (samples_leaf):   0%|          | 0/50 [00:00<?, ?it/s]

...

RandomForestClassifier (n_estimators):   0%|          | 0/9 [00:00<?, ?it/s]

RandomForestClassifier (n_estimators):   0%|          | 0/9 [00:00<?, ?it/s]

RandomForestClassifier (n_estimators):   0%|          | 0/9 [00:00<?, ?it/s]

RandomForestClassifier (n_estimators):   0%|          | 0/9 [00:00<?, ?it/s]

Лучшая модель: RandomForestClassifier(n_estimators=150, max_depth=5)
F1_SCORE: 0.8844221105527638


In [16]:
# Обучаю лучшую модель
best_model.fit(features_train, target_train)
# Вывожу финальную оценку модели
f1_score(target_test, best_model.predict(features_test))

0.9719626168224299

Лучашя модель проходит пороговое значение.

**Вывод раздела.** В данном разделе был проведён малый срез данных для удобства работы с батчами, проведена токенизация, получены эмбединги, разделены признаки и обучены модели и выбрана наилучшая модель. Модель проходит необходимый порог.

## Выводы
Целью данной работы было обучить модель классифицировать комментарии на позитивные и негативные для интернет-магазина «Викишоп». Для достижения данной цели были выполнены следующие задачи:
+ подготовлены и загружены данные;
+ просмотрена основная информация о данных;
+ токенизирован текст;
+ получены эмбединги;
+ данные разделены на тренировачную и тестовую выборки в соотношении 3:1:1;
+ перебором гиперпараметров найдена лучшая модель;
+ лучшая модель прошла порог.

В результате данной работы получены две модели машинного обучения, которые проходят необходимый порог `f1_score` в 0.75, а значит цель работы была достигнута.